<a href="https://colab.research.google.com/github/anandt555/BigData-Practice/blob/main/cifar10c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download and extract CIFAR-10C dataset
!wget -O CIFAR-10-C.tar https://zenodo.org/record/2535967/files/CIFAR-10-C.tar?download=1
!mkdir -p ./data/CIFAR-10-C
!tar -xvf CIFAR-10-C.tar -C ./data/CIFAR-10-C

# Verify extraction
!ls ./data/CIFAR-10-C

--2024-07-09 16:38:56--  https://zenodo.org/record/2535967/files/CIFAR-10-C.tar?download=1
Resolving zenodo.org (zenodo.org)... 188.185.79.172, 188.184.98.238, 188.184.103.159, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/2535967/files/CIFAR-10-C.tar [following]
--2024-07-09 16:38:56--  https://zenodo.org/records/2535967/files/CIFAR-10-C.tar
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 2918471680 (2.7G) [application/octet-stream]
Saving to: ‘CIFAR-10-C.tar’

CIFAR-10-C.tar      100%[===================>]   2.72G  29.1MB/s    in 96s     

2024-07-09 16:40:33 (28.9 MB/s) - ‘CIFAR-10-C.tar’ saved [2918471680/2918471680]

CIFAR-10-C/
CIFAR-10-C/fog.npy
CIFAR-10-C/jpeg_compression.npy
CIFAR-10-C/zoom_blur.npy
CIFAR-10-C/speckle_noise.npy
CIFAR-10-C/glass_blur.npy
CIFAR-10-C/spatter.npy
CIFAR-10-C/shot_noise.npy
CIFAR-10-C/de

# Evidential Deep Learning with Custom Loss - CIFAR-10C (Gaussian Noise)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch.nn.functional as F
from PIL import Image
import os

# Download and extract CIFAR-10C dataset
# !wget -O CIFAR-10-C.tar https://zenodo.org/record/2535967/files/CIFAR-10-C.tar?download=1
# !mkdir -p ./data/CIFAR-10-C
# !tar -xvf CIFAR-10-C.tar -C ./data/CIFAR-10-C

# # Verify extraction
# !ls ./data/CIFAR-10-C

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Download CIFAR-10 dataset for training
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define CIFAR-10C dataset class for testing
class CIFAR10C(torch.utils.data.Dataset):
    def __init__(self, root, corruption_type='gaussian_noise', transform=None):
        self.root = root
        self.transform = transform
        self.corruption_type = corruption_type
        self.data_path = os.path.join(root, 'CIFAR-10-C/CIFAR-10-C', f'{corruption_type}.npy')
        self.labels_path = os.path.join(root, 'CIFAR-10-C/CIFAR-10-C', 'labels.npy')

        self.data = np.load(self.data_path)
        self.labels = np.load(self.labels_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img, label = self.data[idx], self.labels[idx]
        img = Image.fromarray(img)
        if self.transform:
            img = self.transform(img)
        return img, label

# Define transformation for CIFAR-10C
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10C dataset for testing
test_dataset_cifar10c = CIFAR10C(root='./data', corruption_type='gaussian_noise', transform=test_transform)
test_loader_cifar10c = DataLoader(test_dataset_cifar10c, batch_size=64, shuffle=False)

# LeNet model with Dirichlet distribution for Evidential Deep Learning
class LeNetDirichletCIFAR(nn.Module):
    def __init__(self):
        super(LeNetDirichletCIFAR, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(8*8*64, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 8*8*64)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        alpha = torch.abs(x) + 1

        u = 10 / torch.sum(alpha, dim=1, keepdim=True)

        prob = alpha / torch.sum(alpha, 1, keepdim=True)

        return prob, u, alpha

# Define custom loss function and KL divergence
def KL(alpha, num_classes=10):
    one = torch.ones((1, num_classes), dtype=torch.float32)
    S = torch.sum(alpha, dim=1, keepdim=True)

    kl = torch.lgamma(S) - torch.sum(torch.lgamma(alpha), dim=1, keepdim=True) + \
         torch.sum(torch.lgamma(one), dim=1, keepdim=True) - torch.lgamma(torch.sum(one, dim=1, keepdim=True)) + \
         torch.sum((alpha - one) * (torch.digamma(alpha) - torch.digamma(S)), dim=1, keepdim=True)

    return kl

def custom_loss(y_true, output):
    epochs = [1]

    y_evidence = F.relu(output)
    alpha = y_evidence + 1
    S = torch.sum(alpha, dim=1, keepdim=True)
    p = alpha / S

    err = torch.sum(torch.pow((y_true - p), 2), dim=1, keepdim=True)
    var = torch.sum(alpha * (S - alpha) / (S * S * (S + 1)), dim=1, keepdim=True)

    l = torch.sum(err + var, dim=1, keepdim=True)

    kl = torch.min(torch.tensor(1.0), torch.tensor(epochs[0] / 50)) * torch.sum(KL((1 - y_true) * (alpha) + y_true))
    return torch.sum(l + kl)

# Initialize model, optimizer
lenet_dirichlet_CIFAR = LeNetDirichletCIFAR()
optimizer_dirichlet = optim.Adam(lenet_dirichlet_CIFAR.parameters(), lr=0.001)

# Training loop
for epoch in range(30):
    lenet_dirichlet_CIFAR.train()
    for inputs, labels in train_loader:
        optimizer_dirichlet.zero_grad()
        outputs, _, _ = lenet_dirichlet_CIFAR(inputs)
        loss = custom_loss(F.one_hot(labels, num_classes=10).float(), outputs)
        loss.backward()
        optimizer_dirichlet.step()

    # Evaluation on training set
    lenet_dirichlet_CIFAR.eval()
    correct_train = 0
    total_train = 0
    with torch.no_grad():
        for inputs, labels in train_loader:
            outputs, uncertainty_train, _ = lenet_dirichlet_CIFAR(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

    # Evaluation on CIFAR-10C (testing set)
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader_cifar10c:
            outputs, uncertainty_test, _ = lenet_dirichlet_CIFAR(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    # Print metrics
    print('epoch %d - training accuracy: %2.4f \t training uncertainty: %2.4f \t testing accuracy: %2.4f \t testing uncertainty: %2.4f' %
          (epoch+1, correct_train / total_train, uncertainty_train.mean(), correct_test / total_test, uncertainty_test.mean()))


Files already downloaded and verified
epoch 1 - training accuracy: 0.6120 	 training uncertainty: 0.4598 	 testing accuracy: 0.5635 	 testing uncertainty: 0.4690
epoch 2 - training accuracy: 0.6405 	 training uncertainty: 0.3663 	 testing accuracy: 0.6074 	 testing uncertainty: 0.4061
epoch 3 - training accuracy: 0.7196 	 training uncertainty: 0.4099 	 testing accuracy: 0.6195 	 testing uncertainty: 0.4013
epoch 4 - training accuracy: 0.7507 	 training uncertainty: 0.3117 	 testing accuracy: 0.6315 	 testing uncertainty: 0.3089
epoch 5 - training accuracy: 0.7918 	 training uncertainty: 0.3082 	 testing accuracy: 0.6649 	 testing uncertainty: 0.3145
epoch 6 - training accuracy: 0.8056 	 training uncertainty: 0.1925 	 testing accuracy: 0.6578 	 testing uncertainty: 0.3032
epoch 7 - training accuracy: 0.8225 	 training uncertainty: 0.2516 	 testing accuracy: 0.6657 	 testing uncertainty: 0.2870
epoch 8 - training accuracy: 0.8384 	 training uncertainty: 0.3509 	 testing accuracy: 0.6721 

# Evidential Deep Learning + Mixup with Custom Loss - CIFAR-10C (Gaussian Noise)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch.nn.functional as F
from PIL import Image
import os

# Download CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset_cifar10 = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader_cifar10 = DataLoader(dataset=test_dataset_cifar10, batch_size=64, shuffle=False)

# Define CIFAR-10C dataset class for testing
class CIFAR10C(Dataset):
    def __init__(self, root, corruption_type='gaussian_noise', transform=None):
        self.root = root
        self.transform = transform
        self.corruption_type = corruption_type
        self.data_path = os.path.join(root, 'CIFAR-10-C/CIFAR-10-C', f'{corruption_type}.npy')
        self.labels_path = os.path.join(root, 'CIFAR-10-C/CIFAR-10-C', 'labels.npy')

        self.data = np.load(self.data_path)
        self.labels = np.load(self.labels_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img, label = self.data[idx], self.labels[idx]
        img = Image.fromarray(img)
        if self.transform:
            img = self.transform(img)
        return img, label

# Define transformation for CIFAR-10C
test_transform_cifar10c = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10C dataset for testing
test_dataset_cifar10c = CIFAR10C(root='./data', corruption_type='gaussian_noise', transform=test_transform_cifar10c)
test_loader_cifar10c = DataLoader(test_dataset_cifar10c, batch_size=64, shuffle=False)

# LeNet model with Dirichlet distribution for Evidential Deep Learning
class LeNetDirichletCIFARMixup(nn.Module):
    def __init__(self):
        super(LeNetDirichletCIFARMixup, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(8*8*64, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 8*8*64)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        alpha = torch.abs(x) + 1

        u = 10 / torch.sum(alpha, dim=1, keepdim=True)

        prob = alpha / torch.sum(alpha, 1, keepdim=True)

        return prob, u, alpha

# Updated mixup function for Evidential Deep Learning
def mixup_data(x, y, alpha=0.1):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size()[0]
    index = torch.randperm(batch_size)
    mixed_x = lam * x + (1 - lam) * x[index, :]

    # Convert labels to one-hot encoding
    y_onehot = torch.zeros(y.size(0), 10).to(x.device)
    y_onehot.scatter_(1, y.view(-1, 1).long(), 1)
    mixed_y = lam * y_onehot + (1 - lam) * y_onehot[index, :]
    mixed_y = mixed_y.argmax(dim=1)
    return mixed_x, mixed_y

# Define custom loss function and KL divergence
def KL(alpha, num_classes=10):
    one = torch.ones((1, num_classes), dtype=torch.float32)
    S = torch.sum(alpha, dim=1, keepdim=True)

    kl = torch.lgamma(S) - torch.sum(torch.lgamma(alpha), dim=1, keepdim=True) + \
         torch.sum(torch.lgamma(one), dim=1, keepdim=True) - torch.lgamma(torch.sum(one, dim=1, keepdim=True)) + \
         torch.sum((alpha - one) * (torch.digamma(alpha) - torch.digamma(S)), dim=1, keepdim=True)

    return kl

def custom_loss(y_true, output):
    epochs = [1]

    y_evidence = F.relu(output)
    alpha = y_evidence + 1
    S = torch.sum(alpha, dim=1, keepdim=True)
    p = alpha / S

    err = torch.sum(torch.pow((y_true - p), 2), dim=1, keepdim=True)
    var = torch.sum(alpha * (S - alpha) / (S * S * (S + 1)), dim=1, keepdim=True)

    l = torch.sum(err + var, dim=1, keepdim=True)

    kl = torch.min(torch.tensor(1.0), torch.tensor(epochs[0] / 50)) * torch.sum(KL((1 - y_true) * (alpha) + y_true))
    return torch.sum(l + kl)

# Train LeNet model with Dirichlet distribution and mixup for Evidential Deep Learning
lenet_dirichlet_CIFAR_mixup = LeNetDirichletCIFARMixup()
optimizer_dirichlet = optim.Adam(lenet_dirichlet_CIFAR_mixup.parameters(), lr=0.001)

for epoch in range(30):
    lenet_dirichlet_CIFAR_mixup.train()
    for inputs, labels in train_loader:
        inputs, labels = mixup_data(inputs, labels.unsqueeze(1))  # Assuming labels are one-dimensional

        optimizer_dirichlet.zero_grad()
        outputs, _, _ = lenet_dirichlet_CIFAR_mixup(inputs)

        # Cast labels back to Long data type
        labels = labels.squeeze().long()

        loss = custom_loss(F.one_hot(labels, num_classes=10).float(), outputs)  # Using custom loss
        loss.backward()
        optimizer_dirichlet.step()

    lenet_dirichlet_CIFAR_mixup.eval()
    correct_train = 0
    total_train = 0
    with torch.no_grad():
        for inputs, labels in train_loader:
            outputs, uncertainty_train , _ = lenet_dirichlet_CIFAR_mixup(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

    correct_test_cifar10c = 0
    total_test_cifar10c = 0
    with torch.no_grad():
        for inputs, labels in test_loader_cifar10c:
            outputs, uncertainty_test_cifar10c , _ = lenet_dirichlet_CIFAR_mixup(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_test_cifar10c += labels.size(0)
            correct_test_cifar10c += (predicted == labels).sum().item()

    print('epoch %d - training accuracy: %2.4f \t training uncertainty: %2.4f \t testing accuracy on CIFAR-10C: %2.4f \t testing uncertainty on CIFAR-10C: %2.4f' %
          (epoch+1, correct_train / total_train, uncertainty_train.mean(), correct_test_cifar10c / total_test_cifar10c, uncertainty_test_cifar10c.mean()))


Files already downloaded and verified
Files already downloaded and verified
epoch 1 - training accuracy: 0.5806 	 training uncertainty: 0.4947 	 testing accuracy on CIFAR-10C: 0.5613 	 testing uncertainty on CIFAR-10C: 0.4676
epoch 2 - training accuracy: 0.6825 	 training uncertainty: 0.3492 	 testing accuracy on CIFAR-10C: 0.6192 	 testing uncertainty on CIFAR-10C: 0.3683
epoch 3 - training accuracy: 0.7205 	 training uncertainty: 0.3969 	 testing accuracy on CIFAR-10C: 0.6288 	 testing uncertainty on CIFAR-10C: 0.3763
epoch 4 - training accuracy: 0.7400 	 training uncertainty: 0.3862 	 testing accuracy on CIFAR-10C: 0.6273 	 testing uncertainty on CIFAR-10C: 0.3217
epoch 5 - training accuracy: 0.7795 	 training uncertainty: 0.3534 	 testing accuracy on CIFAR-10C: 0.6655 	 testing uncertainty on CIFAR-10C: 0.3249
epoch 6 - training accuracy: 0.7931 	 training uncertainty: 0.3866 	 testing accuracy on CIFAR-10C: 0.6623 	 testing uncertainty on CIFAR-10C: 0.3173
epoch 7 - training accur

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch.nn.functional as F
from PIL import Image
import os

# Download and extract CIFAR-10C dataset
# !wget -O CIFAR-10-C.tar https://zenodo.org/record/2535967/files/CIFAR-10-C.tar?download=1
# !mkdir -p ./data/CIFAR-10-C
# !tar -xvf CIFAR-10-C.tar -C ./data/CIFAR-10-C

# # Verify extraction
# !ls ./data/CIFAR-10-C

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Download CIFAR-10 dataset for training
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define CIFAR-10C dataset class for testing
class CIFAR10C(torch.utils.data.Dataset):
    def __init__(self, root, corruption_type='motion_blur', transform=None):
        self.root = root
        self.transform = transform
        self.corruption_type = corruption_type
        self.data_path = os.path.join(root, 'CIFAR-10-C/CIFAR-10-C', f'{corruption_type}.npy')
        self.labels_path = os.path.join(root, 'CIFAR-10-C/CIFAR-10-C', 'labels.npy')

        self.data = np.load(self.data_path)
        self.labels = np.load(self.labels_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img, label = self.data[idx], self.labels[idx]
        img = Image.fromarray(img)
        if self.transform:
            img = self.transform(img)
        return img, label

# Define transformation for CIFAR-10C
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10C dataset for testing
test_dataset_cifar10c = CIFAR10C(root='./data', corruption_type='motion_blur', transform=test_transform)
test_loader_cifar10c = DataLoader(test_dataset_cifar10c, batch_size=64, shuffle=False)

# LeNet model with Dirichlet distribution for Evidential Deep Learning
class LeNetDirichletCIFAR(nn.Module):
    def __init__(self):
        super(LeNetDirichletCIFAR, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(8*8*64, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 8*8*64)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        alpha = torch.abs(x) + 1

        u = 10 / torch.sum(alpha, dim=1, keepdim=True)

        prob = alpha / torch.sum(alpha, 1, keepdim=True)

        return prob, u, alpha

# Define custom loss function and KL divergence
def KL(alpha, num_classes=10):
    one = torch.ones((1, num_classes), dtype=torch.float32)
    S = torch.sum(alpha, dim=1, keepdim=True)

    kl = torch.lgamma(S) - torch.sum(torch.lgamma(alpha), dim=1, keepdim=True) + \
         torch.sum(torch.lgamma(one), dim=1, keepdim=True) - torch.lgamma(torch.sum(one, dim=1, keepdim=True)) + \
         torch.sum((alpha - one) * (torch.digamma(alpha) - torch.digamma(S)), dim=1, keepdim=True)

    return kl

def custom_loss(y_true, output):
    epochs = [1]

    y_evidence = F.relu(output)
    alpha = y_evidence + 1
    S = torch.sum(alpha, dim=1, keepdim=True)
    p = alpha / S

    err = torch.sum(torch.pow((y_true - p), 2), dim=1, keepdim=True)
    var = torch.sum(alpha * (S - alpha) / (S * S * (S + 1)), dim=1, keepdim=True)

    l = torch.sum(err + var, dim=1, keepdim=True)

    kl = torch.min(torch.tensor(1.0), torch.tensor(epochs[0] / 50)) * torch.sum(KL((1 - y_true) * (alpha) + y_true))
    return torch.sum(l + kl)

# Initialize model, optimizer
lenet_dirichlet_CIFAR = LeNetDirichletCIFAR()
optimizer_dirichlet = optim.Adam(lenet_dirichlet_CIFAR.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    lenet_dirichlet_CIFAR.train()
    for inputs, labels in train_loader:
        optimizer_dirichlet.zero_grad()
        outputs, _, _ = lenet_dirichlet_CIFAR(inputs)
        loss = custom_loss(F.one_hot(labels, num_classes=10).float(), outputs)
        loss.backward()
        optimizer_dirichlet.step()

    # Evaluation on training set
    lenet_dirichlet_CIFAR.eval()
    correct_train = 0
    total_train = 0
    with torch.no_grad():
        for inputs, labels in train_loader:
            outputs, uncertainty_train, _ = lenet_dirichlet_CIFAR(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

    # Evaluation on CIFAR-10C (testing set)
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader_cifar10c:
            outputs, uncertainty_test, _ = lenet_dirichlet_CIFAR(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    # Print metrics
    print('epoch %d - training accuracy: %2.4f \t training uncertainty: %2.4f \t testing accuracy: %2.4f \t testing uncertainty: %2.4f' %
          (epoch+1, correct_train / total_train, uncertainty_train.mean(), correct_test / total_test, uncertainty_test.mean()))


Files already downloaded and verified
epoch 1 - training accuracy: 0.6265 	 training uncertainty: 0.5575 	 testing accuracy: 0.5563 	 testing uncertainty: 0.5760
epoch 2 - training accuracy: 0.6939 	 training uncertainty: 0.4426 	 testing accuracy: 0.5510 	 testing uncertainty: 0.4492
epoch 3 - training accuracy: 0.7472 	 training uncertainty: 0.4744 	 testing accuracy: 0.5896 	 testing uncertainty: 0.4292
epoch 4 - training accuracy: 0.7783 	 training uncertainty: 0.5037 	 testing accuracy: 0.5777 	 testing uncertainty: 0.3341
epoch 5 - training accuracy: 0.8083 	 training uncertainty: 0.2774 	 testing accuracy: 0.6119 	 testing uncertainty: 0.4560
epoch 6 - training accuracy: 0.8291 	 training uncertainty: 0.3770 	 testing accuracy: 0.5927 	 testing uncertainty: 0.3859
epoch 7 - training accuracy: 0.8531 	 training uncertainty: 0.2225 	 testing accuracy: 0.5914 	 testing uncertainty: 0.3551
epoch 8 - training accuracy: 0.8598 	 training uncertainty: 0.3122 	 testing accuracy: 0.5860 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch.nn.functional as F
from PIL import Image
import os

# Download CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset_cifar10 = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader_cifar10 = DataLoader(dataset=test_dataset_cifar10, batch_size=64, shuffle=False)

# Define CIFAR-10C dataset class for testing
class CIFAR10C(Dataset):
    def __init__(self, root, corruption_type='motion_blur', transform=None):
        self.root = root
        self.transform = transform
        self.corruption_type = corruption_type
        self.data_path = os.path.join(root, 'CIFAR-10-C/CIFAR-10-C', f'{corruption_type}.npy')
        self.labels_path = os.path.join(root, 'CIFAR-10-C/CIFAR-10-C', 'labels.npy')

        self.data = np.load(self.data_path)
        self.labels = np.load(self.labels_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img, label = self.data[idx], self.labels[idx]
        img = Image.fromarray(img)
        if self.transform:
            img = self.transform(img)
        return img, label

# Define transformation for CIFAR-10C
test_transform_cifar10c = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10C dataset for testing
test_dataset_cifar10c = CIFAR10C(root='./data', corruption_type='motion_blur', transform=test_transform_cifar10c)
test_loader_cifar10c = DataLoader(test_dataset_cifar10c, batch_size=64, shuffle=False)

# LeNet model with Dirichlet distribution for Evidential Deep Learning
class LeNetDirichletCIFARMixup(nn.Module):
    def __init__(self):
        super(LeNetDirichletCIFARMixup, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(8*8*64, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 8*8*64)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        alpha = torch.abs(x) + 1

        u = 10 / torch.sum(alpha, dim=1, keepdim=True)

        prob = alpha / torch.sum(alpha, 1, keepdim=True)

        return prob, u, alpha

# Updated mixup function for Evidential Deep Learning
def mixup_data(x, y, alpha=0.1):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size()[0]
    index = torch.randperm(batch_size)
    mixed_x = lam * x + (1 - lam) * x[index, :]

    # Convert labels to one-hot encoding
    y_onehot = torch.zeros(y.size(0), 10).to(x.device)
    y_onehot.scatter_(1, y.view(-1, 1).long(), 1)
    mixed_y = lam * y_onehot + (1 - lam) * y_onehot[index, :]
    mixed_y = mixed_y.argmax(dim=1)
    return mixed_x, mixed_y

# Define custom loss function and KL divergence
def KL(alpha, num_classes=10):
    one = torch.ones((1, num_classes), dtype=torch.float32)
    S = torch.sum(alpha, dim=1, keepdim=True)

    kl = torch.lgamma(S) - torch.sum(torch.lgamma(alpha), dim=1, keepdim=True) + \
         torch.sum(torch.lgamma(one), dim=1, keepdim=True) - torch.lgamma(torch.sum(one, dim=1, keepdim=True)) + \
         torch.sum((alpha - one) * (torch.digamma(alpha) - torch.digamma(S)), dim=1, keepdim=True)

    return kl

def custom_loss(y_true, output):
    epochs = [1]

    y_evidence = F.relu(output)
    alpha = y_evidence + 1
    S = torch.sum(alpha, dim=1, keepdim=True)
    p = alpha / S

    err = torch.sum(torch.pow((y_true - p), 2), dim=1, keepdim=True)
    var = torch.sum(alpha * (S - alpha) / (S * S * (S + 1)), dim=1, keepdim=True)

    l = torch.sum(err + var, dim=1, keepdim=True)

    kl = torch.min(torch.tensor(1.0), torch.tensor(epochs[0] / 50)) * torch.sum(KL((1 - y_true) * (alpha) + y_true))
    return torch.sum(l + kl)

# Train LeNet model with Dirichlet distribution and mixup for Evidential Deep Learning
lenet_dirichlet_CIFAR_mixup = LeNetDirichletCIFARMixup()
optimizer_dirichlet = optim.Adam(lenet_dirichlet_CIFAR_mixup.parameters(), lr=0.001)

for epoch in range(30):
    lenet_dirichlet_CIFAR_mixup.train()
    for inputs, labels in train_loader:
        inputs, labels = mixup_data(inputs, labels.unsqueeze(1))  # Assuming labels are one-dimensional

        optimizer_dirichlet.zero_grad()
        outputs, _, _ = lenet_dirichlet_CIFAR_mixup(inputs)

        # Cast labels back to Long data type
        labels = labels.squeeze().long()

        loss = custom_loss(F.one_hot(labels, num_classes=10).float(), outputs)  # Using custom loss
        loss.backward()
        optimizer_dirichlet.step()

    lenet_dirichlet_CIFAR_mixup.eval()
    correct_train = 0
    total_train = 0
    with torch.no_grad():
        for inputs, labels in train_loader:
            outputs, uncertainty_train , _ = lenet_dirichlet_CIFAR_mixup(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

    correct_test_cifar10c = 0
    total_test_cifar10c = 0
    with torch.no_grad():
        for inputs, labels in test_loader_cifar10c:
            outputs, uncertainty_test_cifar10c , _ = lenet_dirichlet_CIFAR_mixup(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_test_cifar10c += labels.size(0)
            correct_test_cifar10c += (predicted == labels).sum().item()

    print('epoch %d - training accuracy: %2.4f \t training uncertainty: %2.4f \t testing accuracy on CIFAR-10C: %2.4f \t testing uncertainty on CIFAR-10C: %2.4f' %
          (epoch+1, correct_train / total_train, uncertainty_train.mean(), correct_test_cifar10c / total_test_cifar10c, uncertainty_test_cifar10c.mean()))


Files already downloaded and verified
Files already downloaded and verified
epoch 1 - training accuracy: 0.5787 	 training uncertainty: 0.5134 	 testing accuracy on CIFAR-10C: 0.5148 	 testing uncertainty on CIFAR-10C: 0.5457
epoch 2 - training accuracy: 0.6746 	 training uncertainty: 0.4801 	 testing accuracy on CIFAR-10C: 0.5402 	 testing uncertainty on CIFAR-10C: 0.4881
epoch 3 - training accuracy: 0.7284 	 training uncertainty: 0.4249 	 testing accuracy on CIFAR-10C: 0.5933 	 testing uncertainty on CIFAR-10C: 0.4533
epoch 4 - training accuracy: 0.7695 	 training uncertainty: 0.3467 	 testing accuracy on CIFAR-10C: 0.6068 	 testing uncertainty on CIFAR-10C: 0.4417
epoch 5 - training accuracy: 0.7873 	 training uncertainty: 0.1863 	 testing accuracy on CIFAR-10C: 0.5929 	 testing uncertainty on CIFAR-10C: 0.4021
epoch 6 - training accuracy: 0.8143 	 training uncertainty: 0.3521 	 testing accuracy on CIFAR-10C: 0.6271 	 testing uncertainty on CIFAR-10C: 0.3906
epoch 7 - training accur

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt

# Download CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./data1', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data1', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# LeNet model with Dirichlet distribution for Evidential Deep Learning and mixup
class LeNetDirichletCIFARMixup(nn.Module):
    def __init__(self):
        super(LeNetDirichletCIFARMixup, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(8*8*64, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = x.view(-1, 8*8*64)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        alpha = torch.abs(x) + 1
        u = 10 / torch.sum(alpha, dim=1, keepdim=True)
        prob = alpha / torch.sum(alpha, 1, keepdim=True)

        return prob, u, alpha

# Define custom loss function and KL divergence
def KL(alpha, num_classes=10):
    one = torch.ones((1, num_classes), dtype=torch.float32)
    S = torch.sum(alpha, dim=1, keepdim=True)

    kl = torch.lgamma(S) - torch.sum(torch.lgamma(alpha), dim=1, keepdim=True) + \
         torch.sum(torch.lgamma(one), dim=1, keepdim=True) - torch.lgamma(torch.sum(one, dim=1, keepdim=True)) + \
         torch.sum((alpha - one) * (torch.digamma(alpha) - torch.digamma(S)), dim=1, keepdim=True)

    return kl

def custom_loss(y_true, output):
    epochs = [1]

    y_evidence = torch.relu(output)
    alpha = y_evidence + 1
    S = torch.sum(alpha, dim=1, keepdim=True)
    p = alpha / S

    err = torch.sum(torch.pow((y_true - p), 2), dim=1, keepdim=True)
    var = torch.sum(alpha * (S - alpha) / (S * S * (S + 1)), dim=1, keepdim=True)

    l = torch.sum(err + var, dim=1, keepdim=True)

    kl = torch.min(torch.tensor(1.0), torch.tensor(epochs[0] / 50)) * torch.sum(KL((1 - y_true) * (alpha) + y_true))
    return torch.sum(l + kl)

# Train LeNet model with Dirichlet distribution for Evidential Deep Learning
lenet_dirichlet_CIFAR = LeNetDirichletCIFARMixup()
optimizer_dirichlet = optim.Adam(lenet_dirichlet_CIFAR.parameters(), lr=0.001)

for epoch in range(10):
    lenet_dirichlet_CIFAR.train()
    for inputs, labels in train_loader:
        optimizer_dirichlet.zero_grad()
        outputs, _, _ = lenet_dirichlet_CIFAR(inputs)
        loss = custom_loss(F.one_hot(labels, num_classes=10).float(), outputs)  # Using custom loss
        loss.backward()
        optimizer_dirichlet.step()

# FGSM attack
def fgsm_attack(model, loss_fn, images, labels, epsilon):
    images = images.clone().detach().requires_grad_(True)
    outputs, _, _ = model(images)
    loss = loss_fn(F.one_hot(labels, num_classes=10).float(), outputs)
    model.zero_grad()
    loss.backward()
    data_grad = images.grad.data
    perturbed_image = images + epsilon * data_grad.sign()
    return perturbed_image

# Calculate accuracy and entropy for different perturbation levels
epsilon_values = np.linspace(0, 0.3, 10)
accuracy = []
entropy = []

for epsilon in epsilon_values:
    correct = 0
    total = 0
    total_entropy = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            adv_inputs = fgsm_attack(lenet_dirichlet_CIFAR, custom_loss, inputs, labels, epsilon)
            outputs, _, alpha = lenet_dirichlet_CIFAR(adv_inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            total_entropy += -torch.sum(outputs * torch.log(outputs + 1e-10)).item()

    accuracy.append(correct / total)
    entropy.append(total_entropy / total)

# Plot the graph
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epsilon_values, accuracy, label='Accuracy')
plt.xlabel('Adversarial Perturbation (ε)')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. Adversarial Perturbation')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epsilon_values, entropy, label='Entropy', color='orange')
plt.xlabel('Adversarial Perturbation (ε)')
plt.ylabel('Entropy')
plt.title('Entropy vs. Adversarial Perturbation')
plt.legend()

plt.tight_layout()
plt.show()

Files already downloaded and verified
Files already downloaded and verified


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn